## Data Downloader for ERA5 and GLoFAS data

defines some methods to download era5 data

In [12]:
import os, warnings
import cdsapi

In [9]:
UID = #enter_uid_here
API_key = #enter_key_here

with open(os.path.join(os.path.expanduser('~'), '.cdsapirc'), 'w') as f:
    f.write('url: https://cds.climate.copernicus.eu/api/v2\n')
    f.write(f'key: {UID}:{API_key}')

SyntaxError: invalid syntax (<ipython-input-9-89ad931fed9e>, line 1)

In [32]:
def build_request(kwargs, input_checking=True):                    
                                  
    mandatory_fields = ["product_type", "format", "variable", "year", "month"]
    if not input_checking:
        mandatory_fields = []
    
    assumed_args = {"day":    ["01", "02", "03", "04",
                               "05", "06", "07", "08",
                               "09", "10", "11", "12",
                               "13", "14", "15", "16",
                               "17", "18", "19", "20",
                               "21", "22", "23", "24",
                               "25", "26", "27", "28",
                               "29", "30", "31"],
                "time":      ["00", "01", "02", "03", "04", "05",
                              "06", "07", "08", "09", "10", "11",
                              "12", "13","14", "15", "16", "17",
                              "18", "19", "20","21", "22", "23"]}
    assume_fields = assumed_args.keys()

    # input checks
    for key in mandatory_fields:  # add mandatory arguments
        if key not in kwargs: 
            raise ValueError(key+' not found in arguments, but is a mandatory field.')
    
    if kwargs['base_level'] == 'pressure' and 'pressure_level' not in kwargs:
        raise IOError('base_level is pressure, but pressure_level not in kwargs')
    
    request_name = f"reanalysis-era5-{kwargs.pop('base_level')}-levels"
    request = {}
    for key in mandatory_fields:
        request[key] = kwargs.pop(key)
            
    for key in list(kwargs):  # add optional arguments
        request[key] = kwargs.pop(key)
                                  
    for key in assume_fields:  # assume some arguments if not given
        if key not in request:
            print(key, 'not found in arguments, assuming', key, '=', )
    
    return request
                                  

def cds_optimized_retrieval(save_to_folder, request: dict, time_start='2017:05', time_end='2017:07'):
    c = cdsapi.Client()
    
    dataset_name = f"reanalysis-era5-{request['base_level']}-levels"
    
    # download era5 data with the cdsapi
    # data request efficiency is highest when executed on a monthly basis
    years = range(int(time_start.split(':')[0]), int(time_end.split(':')[0])+1)
    months = range(int(time_start.split(':')[1]), int(time_end.split(':')[1])+1)

    # loop over time range
    for y in years:
        for m in months:
            # start a request for one month
            # only execute if file does not exist
            request = build_request(request)         
            save_to_filename = f'{save_to_folder}/{dataset_name}_{request["variable"]}_{y}_{"%:02d".format(m)}.nc'
            
            if not os.path.isfile(save_to_filename):
                print(request); pass #c.retrieve(dataset_name, request, save_to_filename)

In [33]:
request = dict(base_level='pressure', form='netcdf', area='50/7/47/20',
               variable='geopotential', pressure_level='700', time_start='2017:05', time_end='2017:07')


save_to_folder='/raid/home/srvx7/lehre/users/a1254888/ipython/ml_flood/data/'
cds_optimized_retrieval(save_to_folder, request, time_start='2017:05', time_end='2017:07')

ValueError: product_type not found in arguments, but is a mandatory field.

### download request for the data of interest on pressure levels

In [ ]:
# define areas of interest
area_dict = {'danube': '50/7/47/20',
             'asia': '55/-140/0/35',
             'usa': '50/-125/25/-70'
            }
# choose area: 'danube', 'asia', 'usa'
area = area_dict['danube']

# variables
variable = ['geopotential', 'temperature']#, 'specific humidity']

# pressure levels
base_level = 'pressure' # 'pressure' or 'single'
pressure_level = ['850', '700', '500']


# define time range: start end in the format YYYY:MM
time_start = '1981:01' # full range start: '1981:01'
time_end = '2017:12' # full range end: '2017:12'

# create savename string
variablestr = "_".join([x.replace(' ', '_') for x in variable])
pressure_levelstr = "_".join([x for x in pressure_level])
savename = f'era5_{variablestr}_{pressure_levelstr}'

# retrieve data
era5_retrieval(savename=savename, area=area, base_level=base_level, variable=variable,
               pressure_level=pressure_level, time_start=time_start, time_end=time_end)